In [ ]:
import torch
import torch.utils.data as data
from torchvision import datasets, transforms

import os
import os.path

import json

from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp -r /content/drive/'My Drive'/APS360/combined_model/ /content/.

In [ ]:
!pip install gdown
!gdown --id 1cN4dAXKQW-cZAqcaKXXHoLqzt9YEOBXr --output combined_model_test.zip 

Downloading...
From: https://drive.google.com/uc?id=1F03OZ0UhaeMkbsEWYA1UHfeU2jRDLip7
To: /content/combined_model_test.zip
86.9MB [00:01, 52.1MB/s]


In [ ]:
!unzip -q combined_model_test.zip

In [ ]:
!unzip -q combined_model_select.zip

In [ ]:
# Note, newest version of opencv may have issues, please use
!pip uninstall opencv-python
!pip install opencv-python==4.1.2.30
# to get opencv

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 28.3MB 148kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [23]:

import glob
import os
import random
import timeit

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image, ImageDraw, ImageFont

torch.manual_seed(1)

# Class Wrapper for YOLO bbox generator and localization
class SignDetector(object):
    def __init__(self, cfg_path, weights_path):
        """Constructor reads a YOLO cfg_path and weights_path

        Parameters
        ----------
        cfg_path : string
            YOLO cfg filepath
        weights_path : string
            YOLO weights filepath
        """
        self.cfg_path = os.path.abspath(cfg_path)
        self.weights_path = os.path.abspath(weights_path)

        self.net = cv2.dnn.readNet(self.weights_path, self.cfg_path)
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]

    def __call__(self, *input, **kwargs):
        """Enables forward function to be called as Object(input)

        Returns
        -------
        img : cv2 image
            input image to generate bbox for.
        """
        return self.forward(*input, **kwargs)

    def forward(self, img):
        """Calculates the bbox for a cv2 image

        Parameters
        ----------
        img : cv2 image
            input image

        Returns
        -------
        list of tuples
            list of tuples, each tuple represents a bbox as (x_top_left, y_top_left, width, height)
        """
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        height, width, channels = img.shape

        # Create a blob which acts as input to our model
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        # Run the Darknet algorithm
        start = timeit.default_timer()
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)
        stop = timeit.default_timer()
        #print("Yolo took {} to run".format(stop-start))

        # Process the output from darknet.
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.3:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append((x, y, w, h))
                    confidences.append(float(confidence))

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        return [boxes[x[0]] for x in indexes]

class SignClassifier(nn.Module):
    def __init__(self):
        super(SignClassifier, self).__init__()
        self.name = "signclassifierAdvancedWithDropout"
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096,1000)
        self.fc3 = nn.Linear(1000,75)
        self.dropout1 = nn.Dropout(0.5) # 50% dropout rate
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 256 * 6 * 6) #flatten feature data
        x = F.relu(self.fc1(self.dropout1(x)))
        x = F.relu(self.fc2(self.dropout2(x)))
        x = self.fc3(self.dropout3(x))
        return x

class TrafficSignRecognizer(object):
    def __init__(self, cfg_path, weights_path, classifier_state, index_to_classes, stride=256, size=416, use_cuda=False):
        """Constructor for the model which does bbox detection and classification of traffic signs

        Parameters
        ----------
        cfg_path : string
            path to YOLO cfg file
        weights_path : string
            path to YOLO weights file
        classifier_state : string
            path to classifier torch model save file
        index_to_classes : string
            path to file containing mapping of classifier output index to classname
        use_cuda : bool, optional
            Turn on CUDA for model, currently not working, by default False
        """
        self.stride = stride
        self.size = size

        self.detector = SignDetector(cfg_path, weights_path)
        self.classifier = SignClassifier()

        # Load the pretrained state into our classifier and set to eval mode to ignore dropout
        state = torch.load(classifier_state, map_location=torch.device('cpu'))
        self.classifier.load_state_dict(state)
        self.classifier.eval()

        # alexnet pretrained model for feature extraction
        import torchvision.models
        self.alexnet = torchvision.models.alexnet(pretrained=True)

        # map the output of classifier to a classname
        self.index_to_class = list()
        with open(index_to_classes, "r") as fp:
            for line in fp:
                self.index_to_class.append(line.strip())

        # define the transformations for our classifier
        self.transform = transforms.Compose(
                [transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def __get_bboxes(self, img):
        # img = Image.open(image_file)
        bboxes = self.detector(img)
        res = []
        for bbox in bboxes:
            x1 = bbox[0]
            y1 = bbox[1]
            x2 = x1 + bbox[2]
            y2 = y1 + bbox[3]
            res.append(torch.tensor((x1, y1, x2, y2)))
        return res

    def __break_up_image(self, img):
        # Calc Image size
        imgwidth, imgheight = img.size
        index_to_bbox = []

        # Crop
        index = 0
        for i in range(0, imgheight, self.stride):
            for j in range(0, imgwidth, self.stride):
                # Out of bounds!
                if j+self.size >= imgwidth or i+self.size >= imgheight:
                    continue
                crop_box = (j, i, j+self.size, i+self.size)
                cropped = img.crop(crop_box)
                index_to_bbox.append((torch.tensor(crop_box), cropped))
                index +=1
        return index_to_bbox

    def __call__(self, *input, **kwargs):
        return self.forward(*input, **kwargs)

    def forward(self, img, iou_threshold=0.25):
        img = img.convert('RGB')
        # Map the crop index to bbox based on original image that specifies that crop
        index_to_bbox = self.__break_up_image(img)

        yolobbox_tensor = list()
        for index in range(len(index_to_bbox)):
            crop_bbox, crop_img = index_to_bbox[index]
            # Get the bbox from yolo
            yolo_bboxes = self.__get_bboxes(crop_img)

            # Change a relative bbox to absolute bbox
            for yolo_bbox in yolo_bboxes:
                abs_bbox = [yolo_bbox[0]+crop_bbox[0],
                            yolo_bbox[1]+crop_bbox[1],
                            yolo_bbox[2]+crop_bbox[0],
                            yolo_bbox[3]+crop_bbox[1]]

                #Check if box is >= 150 pixels
                abs_width = (abs_bbox[2] - abs_bbox[0]) * (abs_bbox[3] - abs_bbox[1])
                if (abs_width >= 150):
                  yolobbox_tensor.append(abs_bbox)



        if not yolobbox_tensor:
            print("No bounding boxes")
            return

        # Convert list of bbox to torch tensor and run nms on the bboxes to merge
        yolobbox_tensor = torch.tensor(yolobbox_tensor, dtype=torch.float)
        kept_yolobbox = torchvision.ops.nms(yolobbox_tensor, torch.tensor([1.0] * yolobbox_tensor.shape[0]), iou_threshold)

        # kept is a list of indexes that correspond to a bbox in bbox_tensor
        # We want to generate a return which is: [(bbox, class, probability)]
        res = list()
        for index in kept_yolobbox:
            bbox = [int(x) for x in yolobbox_tensor[index]]
            cropped = img.crop(bbox)

            # Prepare the cropped yolo bbox for input to our classifier
            cropped = self.transform(cropped).float()
            cropped = cropped.unsqueeze(0)
            features = self.alexnet.features(cropped)
            features = torch.from_numpy(features.detach().numpy())

            # Run the classifier on the cropped yolo bbox
            output = F.softmax(self.classifier(features), dim=1)
            prediction = torch.argmax(output)
            probability = torch.max(output)
            res.append((int(prediction), float(probability), bbox))
        return res

    def get_class_name_from_index(self, index):
        return self.index_to_class[index]

    def annotate_image(self, img, annotation_items):
        img = img.convert('RGB')
        drawing = ImageDraw.Draw(img)

        for prediction, probability, bbox in annotation_items:
            label = self.get_class_name_from_index(prediction)
            drawing.rectangle(bbox, outline=(252, 3, 177), width=2)
            try:
                font = ImageFont.truetype("/content/combined_model/font.ttf", 20)
            except:
                raise Exception("Current directory missing font.ttf for use in drawing labels on final image.")
            drawing.text((bbox[0], bbox[1]-25), label, font=font,fill='white',stroke_width=1,stroke_fill='black')
        return img




In [ ]:
########################################
# Example Run + How to Use

# YOLO config and pretrained weights
cfg_filename = "/content/combined_model/yolov3_training.cfg"
weights_filename = "/content/combined_model/yolov3_training_5000.weights"
state_filename = "/content/combined_model/model_signclassifierAdvancedWithDropout_bs256_lr3e-05_wd0.0001_dropout50_epoch66"
index_filename = "/content/combined_model/index_to_class.txt"
input_image = "/content/combined_model_test/-4gtcRinmMXt0PgT-ROzMg.jpg"

start = timeit.default_timer()

# Run the merged model
model = TrafficSignRecognizer(cfg_filename, weights_filename, state_filename, index_filename, stride=416)
img = Image.open(input_image)
output = model(img)

# Debug
print(output)

# Save the output annotated file
#if (len(output) > 0):
#  img = model.annotate_image(img, output)
#out_path = os.path.join("combined_model_output", "annotated_" + os.path.split(input_image)[1])
#img.save(out_path)




stop = timeit.default_timer()
print("Took {} to run".format(stop-start))
########################################

[557, 519, 567, 535]
[1139, 535, 1161, 555]
[(22, 0.1720220297574997, [557, 519, 567, 535]), (43, 0.9911988377571106, [1139, 535, 1161, 555])]
Took 15.717458602997795 to run


In [ ]:
index_to_class = list()
with open(index_filename, "r") as fp:
  for line in fp:
    index_to_class.append(line.strip())

#print (index_to_class[output[0][0]])
test_dict = {}
for i in output:
  if index_to_class[i[0]] in test_dict:
    count = test_dict[index_to_class[i[0]]]
    test_dict[index_to_class[i[0]]] = count + 1
  else:
    test_dict[index_to_class[i[0]]] = 1

print (test_dict)


**Run Combined Test**

In [ ]:
def calc_combined_acc (test_list):

  TP = 0
  FP = 0
  FN = 0

  class_statistic_dict = {}

  cfg_filename = "/content/combined_model/yolov3_training.cfg"
  weights_filename = "/content/combined_model/yolov3_training_5000.weights"
  state_filename = "/content/combined_model/model_signclassifierAdvancedWithDropout_bs256_lr3e-05_wd0.0001_dropout50_epoch66"
  index_filename = "/content/combined_model/index_to_class.txt"

  index_to_class = list()
  with open("/content/combined_model/index_to_class_full_name.txt", "r") as fp:
    for line in fp:
      index_to_class.append(line.strip())

  for i in index_to_class:
    class_statistic_dict[i + "_TP"] = 0
    class_statistic_dict[i + "_FP"] = 0
    class_statistic_dict[i + "_FN"] = 0

  for image_file, txt_file in test_list:
    img = Image.open(image_file)
    label_dict = {}
    pred_dict = {}

    #Get labels
    with open(txt_file, "r") as txt:
        for line in txt:
          line_split = line.split()
          #print (line_split)
          if line_split[0] in label_dict:
            count = label_dict[line_split[0]]
            label_dict[line_split[0]] = count + 1
          else:
            label_dict[line_split[0]] = 1

  
   ###### Feed image into model here #########
    # Model outputs a dictionary - with key = sign label, and value = number occurance

    #pred_dict = model ...
    #pred_dict = label_dict.copy()

    start = timeit.default_timer()

    # Run the merged model
    model = TrafficSignRecognizer(cfg_filename, weights_filename, state_filename, index_filename, stride=416)
    output = model(img)

    # Debug
    print(output)

    # Save the output annotated file
    if (output != None):
      img = model.annotate_image(img, output)
      #Get predictions
      for i in output:
        #Check if output is >= 0.5
        if (i[1] > 0.3):
          if index_to_class[i[0]] in pred_dict:
            count = pred_dict[index_to_class[i[0]]]
            pred_dict[index_to_class[i[0]]] = count + 1
          else:
            pred_dict[index_to_class[i[0]]] = 1
    out_path = os.path.join("combined_model_output", "annotated_" + os.path.split(image_file)[1])
    img.save(out_path)


    

    stop = timeit.default_timer()
    print("Took {} to run".format(stop-start))

  

    ############################################
  
    for x in pred_dict:
      if (x in label_dict):
        if (pred_dict[x] == label_dict[x]):
          TP = TP + pred_dict[x]
          class_statistic_dict[x + "_TP"] = class_statistic_dict[x + "_TP"] + pred_dict[x]

        elif (pred_dict[x] < label_dict[x]):
          #We predicted less than expected
          TP = TP + pred_dict[x]
          class_statistic_dict[x + "_TP"] = class_statistic_dict[x + "_TP"] + pred_dict[x]

          FN = FN + label_dict[x] - pred_dict[x]
          class_statistic_dict[x + "_FN"] = class_statistic_dict[x + "_FN"] + label_dict[x] - pred_dict[x]

        elif (pred_dict[x] > label_dict[x]):
          #We predicted more than expected
          TP =  TP + label_dict[x]
          class_statistic_dict[x + "_TP"] = class_statistic_dict[x + "_TP"] + label_dict[x]

          FP = FP + pred_dict[x] - label_dict[x]
          class_statistic_dict[x + "_FP"] = class_statistic_dict[x + "_FP"] + pred_dict[x] - label_dict[x]

        #Remove the key from the label_dict
        del label_dict[x]
      else:
        #We predicted something that was not there
        FP = FP + pred_dict[x]
        class_statistic_dict[x + "_FP"] = class_statistic_dict[x + "_FP"] + pred_dict[x]

    #Count remaining signs that we missed
    for x in label_dict:
      FN = FN + label_dict[x]
      class_statistic_dict[x + "_FN"] = class_statistic_dict[x + "_FN"] + label_dict[x]

    print ("TP, FN, FP: ", TP, " ", FN, " ", FP)

  #Avoid dividing by 0 - should be impossible
  precision = 0
  if ((TP + FP ) > 0):
    precision = TP / (TP + FP)

  recall = 0
  if ((TP + FN) > 0):
    recall = TP / (TP + FN)
  
  acc = 0
  if ((TP + FN + FP) > 0):
    acc = TP / (TP + FN + FP)

  

  print ("Precision: ", precision)
  print ("Recall: ", recall)
  print ("Total Accuracy: ", acc)

  return class_statistic_dict

In [24]:
import glob

!rm -rf combined_model_output/
!mkdir combined_model_output/

label_list = sorted(glob.glob("/content/combined_model_test/*.txt"))
images_list = sorted(glob.glob("/content/combined_model_test/*.jpg"))
result_dict = calc_combined_acc (zip(images_list, label_list))


[(22, 0.17202192544937134, [557, 519, 567, 535]), (43, 0.9911988377571106, [1139, 535, 1161, 555])]
Took 16.51930267300031 to run
TP, FN, FP:  1   1   0
[(2, 0.8632381558418274, [403, 640, 414, 666]), (4, 0.5482599139213562, [416, 639, 424, 666]), (34, 0.9997616410255432, [980, 608, 1010, 637])]
Took 15.35829008500059 to run
TP, FN, FP:  2   1   2
[(58, 0.9997319579124451, [558, 635, 581, 656]), (58, 0.9992503523826599, [770, 614, 789, 636]), (1, 0.9999995231628418, [959, 589, 997, 636]), (1, 0.9999994039535522, [1024, 577, 1073, 641]), (1, 0.9999960660934448, [1142, 558, 1208, 633])]
Took 14.435903274999873 to run
TP, FN, FP:  4   1   5
[(48, 1.0, [82, 293, 157, 393]), (43, 0.9938702583312988, [164, 279, 271, 342]), (48, 0.5298095941543579, [694, 553, 705, 567])]
Took 14.55797336699925 to run
TP, FN, FP:  5   1   7
Precision:  0.4166666666666667
Recall:  0.8333333333333334
Total Accuracy:  0.38461538461538464


In [ ]:
print (result_dict)

{'complementary--both-directions--g1_TP': 0, 'complementary--both-directions--g1_FP': 1, 'complementary--both-directions--g1_FN': 0, 'complementary--chevron-left--g1_TP': 5, 'complementary--chevron-left--g1_FP': 7, 'complementary--chevron-left--g1_FN': 0, 'complementary--chevron-right--g1_TP': 6, 'complementary--chevron-right--g1_FP': 9, 'complementary--chevron-right--g1_FN': 0, 'complementary--obstacle-delineator--g1_TP': 1, 'complementary--obstacle-delineator--g1_FP': 0, 'complementary--obstacle-delineator--g1_FN': 1, 'complementary--obstacle-delineator--g2_TP': 1, 'complementary--obstacle-delineator--g2_FP': 4, 'complementary--obstacle-delineator--g2_FN': 1, 'complementary--tow-away-zone--g1_TP': 0, 'complementary--tow-away-zone--g1_FP': 0, 'complementary--tow-away-zone--g1_FN': 0, 'information--airport--g2_TP': 0, 'information--airport--g2_FP': 0, 'information--airport--g2_FN': 0, 'information--bike-route--g1_TP': 1, 'information--bike-route--g1_FP': 0, 'information--bike-route--g1

In [ ]:
def get_sensitivity (result_dict, sign_name):

  nominator = result_dict[sign_name + "_TP"]
  denominator = (result_dict[sign_name + "_TP"] + result_dict[sign_name + "_FN"])

  result = 0
  if (denominator > 0):
    result = nominator / denominator

  return result

  return result_dict[sign_name + "_TP"] / (result_dict[sign_name + "_TP"] + result_dict[sign_name + "_FN"])

def get_precision (result_dict, sign_name):

  nominator = result_dict[sign_name + "_TP"]
  denominator = (result_dict[sign_name + "_TP"] + result_dict[sign_name + "_FP"])

  result = 0
  if (denominator > 0):
    result = nominator / denominator
    
  return result

index_to_class = list()
with open("/content/combined_model/index_to_class_full_name.txt", "r") as fp:
  for line in fp:
    index_to_class.append(line.strip())


most_common = ""
most_common_count = 0
least_common = ""
least_common_count = 1000

most_correct = ""
most_correct_num = 0
least_correct = ""
least_correct_num = 1000

for i in index_to_class:
  TP = result_dict[i + "_TP"]
  FP = result_dict[i + "_FP"]
  FN = result_dict[i + "_FN"]

  real_count = TP + FN

  if (real_count > most_common_count):
    most_common = i
    most_common_count = real_count
  
  if (real_count < least_common_count):
    least_common = i
    least_common_count = real_count
  
  correct = TP - FP - FN

  if (correct > most_correct_num):
    most_correct = i
    most_correct_num = correct
  if (correct < least_correct_num):
    least_correct = i
    least_correct_num = correct

print ("most_common: ", most_common, "count: ", most_common_count,
       "sensitivity: ", get_sensitivity(result_dict, most_common),
       "precision: ", get_precision(result_dict, most_common))


print ("least common: ", least_common, "count: ", least_correct_count,
       "sensitivity: ", get_sensitivity(result_dict, least_common),
       "precision: ", get_precision(result_dict, least_common))

print ("most_correct: ", most_correct, "Times Correct: ", most_correct_num,
       "sensitivity: ", get_sensitivity(result_dict, most_correct),
       "precision: ", get_precision(result_dict, most_correct))
print ("least_correct: ", least_correct, "Times Correct: ", least_correct_num,
       "sensitivity: ", get_sensitivity(result_dict, least_correct),
       "precision: ", get_precision(result_dict, least_correct))

most_common:  regulatory--no-entry--g1 count:  14 sensitivity:  0.6428571428571429 precision:  0.6428571428571429
least common:  complementary--both-directions--g1 count:  0 sensitivity:  0 precision:  0.0
most_correct:  regulatory--one-way-left--g3 Times Correct:  2 sensitivity:  1.0 precision:  1.0
least_correct:  warning--y-roads--g1 Times Correct:  -7 sensitivity:  0 precision:  0.0


In [25]:
!zip -q -r /content/drive/'My Drive'/APS360/combined_model/combined_model_output_selected.zip combined_model_output

**Process Test Set**

Use the code below to convert Mapillary images + annotations.
Resizes test set to specified dimensions and removes small signs.
Counts number of occurance for valid signs and saves them in a text file.

In [ ]:
#Get Mapillary Dataset
!pip install gdown
!gdown --id 1J_I1NVNdUGwQs38RfbpL1ih_iyvrLt21 --output test_set.zip 


Downloading...
From: https://drive.google.com/uc?id=1J_I1NVNdUGwQs38RfbpL1ih_iyvrLt21
To: /content/test_set.zip
91.1MB [00:00, 137MB/s] 


In [ ]:
!unzip -q test_set.zip

In [ ]:
#Create a dataset for images/annotations
#Makes it easier to iterate through them
class AnnotationDataSet(data.Dataset):
    def __init__(self, path_to_imgs, path_to_json, transform = None):
        self.path_to_imgs = path_to_imgs
        self.path_to_json = path_to_json
        self.image_ids = os.listdir(path_to_imgs)
        self.transform = transform

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_id = os.path.splitext(img_id)[0]
        img = Image.open(os.path.join(self.path_to_imgs, img_id + ".jpg"))
        json_file = json.load(open(os.path.join(self.path_to_json, img_id + ".json")))


        return img, json_file, img_id

    def __len__(self):
        return len(self.image_ids)

In [ ]:
#Resizes the image to ~1920x1080 for consistency
#Counts the number of signs per image.
#Only counts sign types that are in valid signs

def count_signs (image_file, json_file, name, valid_signs, max_width, max_height, output_dir, discard_dir, min_sign_pixel = 150):

      obj_dict = {}
      imgwidth, imgheight = image_file.size

      #Ratios - Limit the ratios to 1 so we do not enlarge image
      width_ratio = max(1, imgwidth / max_width)
      height_ratio = max(1, imgheight / max_height)

      #Resize by the larger ratio
      resize_ratio = max(width_ratio, height_ratio)

      new_size = (int(imgwidth/resize_ratio), int(imgheight/resize_ratio))
      new_im = image_file.resize(new_size)

      #Bounding box would decrease by ratio^2
      resize_ratio_squared = resize_ratio * resize_ratio

       # Check which signs are large enough to detect
      for i in json_file["objects"]:

          width = i["bbox"]['xmax'] - i["bbox"]['xmin']
          height = i["bbox"]['ymax'] - i["bbox"]['ymin']

          area_resized = width * height / resize_ratio_squared

          if (area_resized < min_sign_pixel):
              continue

          if (i["label"] in valid_signs):
            if (i["label"] in obj_dict):
                count = obj_dict[i["label"]]
                obj_dict[i["label"]] = count + 1

            else:
                obj_dict[i["label"]] = 1



        
      if (len(obj_dict) > 0):

        new_im.save(os.path.join(output_dir, "%s.jpg" % (name)))
        newline = 0  
        with open(os.path.join(output_dir, "%s.txt" % (name)), "w") as fp:
            for x in obj_dict:
              if (newline):
                fp.write("\n")
              str_content = "{0} {1:1d}".format(x, int(obj_dict[x]))
              fp.write(str_content)
              newline = 1
            fp.close()
      else:
        #print ("Image: ", name, " no longer has any detectable signs!")
        new_im.save(os.path.join(discard_dir, "%s.jpg" % (name)))


In [ ]:
import numpy as np
valid_signs = ['regulatory--no-parking--g2', 'regulatory--maximum-speed-limit-55--g2', 'warning--road-narrows-left--g2', 'regulatory--maximum-speed-limit-25--g2', 'warning--pedestrians-crossing--g4', 'warning--roundabout--g2', 'regulatory--keep-left--g2', 'regulatory--do-not-block-intersection--g1', 'regulatory--turn-left--g2', 'regulatory--stop--g1', 'warning--steep-descent--g2', 'complementary--obstacle-delineator--g1', 'regulatory--one-way-right--g3', 'warning--narrow-bridge--g1', 'warning--turn-right--g1', 'warning--t-roads--g2', 'regulatory--wrong-way--g1', 'warning--crossroads--g3', 'regulatory--maximum-speed-limit-30--g3', 'information--disabled-persons--g1', 'regulatory--go-straight--g3', 'regulatory--yield--g1', 'information--gas-station--g1', 'regulatory--no-heavy-goods-vehicles--g2', 'complementary--chevron-right--g1', 'regulatory--no-entry--g1', 'information--highway-exit--g1', 'complementary--tow-away-zone--g1', 'warning--traffic-merges-right--g1', 'regulatory--no-overtaking--g5', 'warning--junction-with-a-side-road-perpendicular-right--g3', 'warning--road-narrows--g2', 'regulatory--no-right-turn--g1', 'warning--turn-left--g1', 'warning--school-zone--g2', 'regulatory--no-straight-through--g1', 'information--hospital--g1', 'regulatory--no-u-turn--g1', 'regulatory--one-way-left--g3', 'warning--road-narrows-right--g2', 'regulatory--maximum-speed-limit-45--g3', 'complementary--both-directions--g1', 'regulatory--no-bicycles--g2', 'warning--winding-road-first-left--g1', 'regulatory--dual-lanes-go-straight-on-right--g1', 'regulatory--triple-lanes-turn-left-center-lane--g1', 'warning--traffic-merges-left--g1', 'warning--traffic-signals--g3', 'warning--y-roads--g1', 'information--telephone--g1', 'complementary--chevron-left--g1', 'regulatory--reversible-lanes--g2', 'warning--winding-road-first-right--g1', 'warning--double-curve-first-right--g2', 'regulatory--maximum-speed-limit-100--g3', 'regulatory--maximum-speed-limit-35--g2', 'regulatory--no-buses--g3', 'information--airport--g2', 'warning--curve-left--g2', 'warning--curve-right--g2', 'regulatory--turn-right--g3', 'warning--junction-with-a-side-road-perpendicular-left--g3', 'warning--double-curve-first-left--g2', 'regulatory--no-turn-on-red--g2', 'regulatory--dual-lanes-go-straight-on-left--g1', 'regulatory--no-left-turn--g2', 'regulatory--road-closed--g2', 'warning--divided-highway-ends--g2', 'warning--height-restriction--g2', 'complementary--obstacle-delineator--g2', 'regulatory--maximum-speed-limit-40--g3', 'information--bike-route--g1', 'regulatory--no-pedestrians--g2', 'regulatory--keep-right--g4', 'regulatory--go-straight-or-turn-left--g2']
test_set = AnnotationDataSet("/content/combined_model_select/images", "/content/combined_model_select/annotations")

for img, json_file, img_id in iter(test_set):
  count_signs (img, json_file, img_id, valid_signs, 1920, 1080, "/content/combined_model_test", "/content/discard", min_sign_pixel = 150)


In [ ]:
!rm -rf /content/combined_model_test/
!mkdir /content/combined_model_test/

In [ ]:
!zip -q -r /content/drive/'My Drive'/APS360/combined_model/combined_model_test.zip combined_model_test   